In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from scipy import stats
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import KFold, StratifiedKFold
import gc
import json
pd.set_option('display.max_columns', 1000)

/kaggle/input/data-science-bowl-2019/specs.csv
/kaggle/input/data-science-bowl-2019/train_labels.csv
/kaggle/input/data-science-bowl-2019/test.csv
/kaggle/input/data-science-bowl-2019/train.csv
/kaggle/input/data-science-bowl-2019/sample_submission.csv


# Objective

* In the last notebook we create our baseline model including a feature selection part. 
* Cohen cappa score of 0.456 (lb) with a local cv score of 0.529
* In this notebook we are going to add more features and remove others that i think they overfitt the train set and then check if our local cv score improve.
* Next, we will check if this improvement aligns with the lb.

# Notes
* Check the distribution of the target variable of the out of folds score and the prediction distribution. A good model should more or less have the same distribution.

In [2]:
def read_data():
    print('Reading train.csv file....')
    train = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv')
    print('Training.csv file have {} rows and {} columns'.format(train.shape[0], train.shape[1]))

    print('Reading test.csv file....')
    test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')
    print('Test.csv file have {} rows and {} columns'.format(test.shape[0], test.shape[1]))

    print('Reading train_labels.csv file....')
    train_labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')
    print('Train_labels.csv file have {} rows and {} columns'.format(train_labels.shape[0], train_labels.shape[1]))

    print('Reading specs.csv file....')
    specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv')
    print('Specs.csv file have {} rows and {} columns'.format(specs.shape[0], specs.shape[1]))

    print('Reading sample_submission.csv file....')
    sample_submission = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv')
    print('Sample_submission.csv file have {} rows and {} columns'.format(sample_submission.shape[0], sample_submission.shape[1]))
    return train, test, train_labels, specs, sample_submission

In [3]:
def encode_title(train, test, train_labels):
    # encode title
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    all_title_event_code = list(set(train["title_event_code"].unique()).union(test["title_event_code"].unique()))
    # make a list with all the unique 'titles' from the train and test set
    list_of_user_activities = list(set(train['title'].unique()).union(set(test['title'].unique())))
    # make a list with all the unique 'event_code' from the train and test set
    list_of_event_code = list(set(train['event_code'].unique()).union(set(test['event_code'].unique())))
    list_of_event_id = list(set(train['event_id'].unique()).union(set(test['event_id'].unique())))
    # make a list with all the unique worlds from the train and test set
    list_of_worlds = list(set(train['world'].unique()).union(set(test['world'].unique())))
    # create a dictionary numerating the titles
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = list(set(train[train['type'] == 'Assessment']['title'].value_counts().index).union(set(test[test['type'] == 'Assessment']['title'].value_counts().index)))
    # replace the text titles with the number titles from the dict
    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)
    train_labels['title'] = train_labels['title'].map(activities_map)
    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))
    # then, it set one element, the 'Bird Measurer (Assessment)' as 4110, 10 more than the rest
    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    # convert text into datetime
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    
    
    return train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code

In [4]:
# this is the function that convert the raw data into processed features
def get_data(user_sample, test_set=False):
    '''
    The user_sample is a DataFrame from train or test where the only one 
    installation_id is filtered
    And the test_set parameter is related with the labels processing, that is only requered
    if test_set=False
    '''
    # Constants and parameters declaration
    last_activity = 0
    
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    
    # new features: time spent in each activity
    last_session_time_sec = 0
    accuracy_groups = {0:0, 1:0, 2:0, 3:0}
    all_assessments = []
    accumulated_accuracy_group = 0
    accumulated_accuracy = 0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0
    accumulated_actions = 0
    counter = 0
    time_first_activity = float(user_sample['timestamp'].values[0])
    durations = []
    last_accuracy_title = {'acc_' + title: -1 for title in assess_titles}
    event_code_count: Dict[str, int] = {ev: 0 for ev in list_of_event_code}
    event_id_count: Dict[str, int] = {eve: 0 for eve in list_of_event_id}
    title_count: Dict[str, int] = {eve: 0 for eve in activities_labels.values()} 
    title_event_code_count: Dict[str, int] = {t_eve: 0 for t_eve in all_title_event_code}
    
    # itarates through each session of one instalation_id
    for i, session in user_sample.groupby('game_session', sort=False):
        # i = game_session_id
        # session is a DataFrame that contain only one game_session
        
        # get some sessions information
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title]
                    
            
        # for each assessment, and only this kind off session, the features below are processed
        # and a register are generated
        if (session_type == 'Assessment') & (test_set or len(session)>1):
            # search for event_code 4100, that represents the assessments trial
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            # then, check the numbers of wins and the number of losses
            true_attempts = all_attempts['event_data'].str.contains('true').sum()
            false_attempts = all_attempts['event_data'].str.contains('false').sum()
            # copy a dict to use as feature template, it's initialized with some itens: 
            # {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
            features = user_activities_count.copy()
            features.update(last_accuracy_title.copy())
            features.update(event_code_count.copy())
            features.update(event_id_count.copy())
            features.update(title_count.copy())
            features.update(title_event_code_count.copy())
            features.update(last_accuracy_title.copy())
            
            # get installation_id for aggregated features
            features['installation_id'] = session['installation_id'].iloc[-1]
            # add title as feature, remembering that title represents the name of the game
            features['session_title'] = session['title'].iloc[0]
            # the 4 lines below add the feature of the history of the trials of this player
            # this is based on the all time attempts so far, at the moment of this assessment
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            # the time spent in the app so far
            if durations == []:
                features['duration_mean'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2] ).seconds)
            # the accurace is the all time wins divided by the all time attempts
            features['accumulated_accuracy'] = accumulated_accuracy/counter if counter > 0 else 0
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            accumulated_accuracy += accuracy
            last_accuracy_title['acc_' + session_title_text] = accuracy
            # a feature of the current accuracy categorized
            # it is a counter of how many times this player was in each accuracy group
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            features.update(accuracy_groups)
            accuracy_groups[features['accuracy_group']] += 1
            # mean of the all accuracy groups of this player
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            accumulated_accuracy_group += features['accuracy_group']
            # how many actions the player has done so far, it is initialized as 0 and updated some lines below
            features['accumulated_actions'] = accumulated_actions
            
            # there are some conditions to allow this features to be inserted in the datasets
            # if it's a test set, all sessions belong to the final dataset
            # it it's a train, needs to be passed throught this clausule: session.query(f'event_code == {win_code[session_title]}')
            # that means, must exist an event_code 4100 or 4110
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
        
        # this piece counts how many actions was made in each event_code so far
        def update_counters(counter: dict, col: str):
                num_of_session_count = Counter(session[col])
                for k in num_of_session_count.keys():
                    x = k
                    if col == 'title':
                        x = activities_labels[k]
                    counter[x] += num_of_session_count[k]
                return counter
            
        event_code_count = update_counters(event_code_count, "event_code")
        event_id_count = update_counters(event_id_count, "event_id")
        title_count = update_counters(title_count, 'title')
        title_event_code_count = update_counters(title_event_code_count, 'title_event_code')

        # counts how many actions the player has done so far, used in the feature of the same name
        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type 
                        
    # if it't the test_set, only the last assessment must be predicted, the previous are scraped
    if test_set:
        return all_assessments[-1]
    # in the train_set, all assessments goes to the dataset
    return all_assessments

In [5]:
def get_train_and_test(train, test):
    compiled_train = []
    compiled_test = []
    for i, (ins_id, user_sample) in tqdm(enumerate(train.groupby('installation_id', sort = False)), total = 17000):
        compiled_train += get_data(user_sample)
    for ins_id, user_sample in tqdm(test.groupby('installation_id', sort = False), total = 1000):
        test_data = get_data(user_sample, test_set = True)
        compiled_test.append(test_data)
    reduce_train = pd.DataFrame(compiled_train)
    reduce_test = pd.DataFrame(compiled_test)
    categoricals = ['session_title']
    return reduce_train, reduce_test, categoricals

In [6]:
def run_feature_selection(reduce_train, reduce_test, usefull_features, new_features):
    kf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
    target = 'accuracy_group'
    oof_pred = np.zeros((len(reduce_train), 4))
    for fold, (tr_ind, val_ind) in enumerate(kf.split(reduce_train, reduce_train[target])):
        print('Fold {}'.format(fold + 1))
        x_train, x_val = reduce_train[usefull_features].iloc[tr_ind], reduce_train[usefull_features].iloc[val_ind]
        y_train, y_val = reduce_train[target][tr_ind], reduce_train[target][val_ind]
        train_set = lgb.Dataset(x_train, y_train, categorical_feature = categoricals)
        val_set = lgb.Dataset(x_val, y_val, categorical_feature = categoricals)

        params = {
            'learning_rate': 0.01,
            'metric': 'multiclass',
            'objective': 'multiclass',
            'num_classes': 4,
            'feature_fraction': 0.7,
            'subsample': 0.7,
            'n_jobs': -1,
            'seed': 50,
            'max_depth': 10
        }

        model = lgb.train(params, train_set, num_boost_round = 100000, early_stopping_rounds = 100, 
                          valid_sets=[train_set, val_set], verbose_eval = 500)
        oof_pred[val_ind] = model.predict(x_val)
    # using cohen_kappa because it's the evaluation metric of the competition
    loss_score = cohen_kappa_score(reduce_train[target], np.argmax(oof_pred, axis = 1), weights = 'quadratic')
    score = loss_score
    usefull_new_features = []
    for i in new_features:
        oof_pred = np.zeros((len(reduce_train), 4))
        evaluating_features = usefull_features + usefull_new_features + [i]
        print('Evaluating {} column'.format(i))
        print('Out best cohen kappa score is : {}'.format(score))
        for fold, (tr_ind, val_ind) in enumerate(kf.split(reduce_train, reduce_train[target])):
            print('Fold {}'.format(fold + 1))
            x_train, x_val = reduce_train[evaluating_features].iloc[tr_ind], reduce_train[evaluating_features].iloc[val_ind]
            y_train, y_val = reduce_train[target][tr_ind], reduce_train[target][val_ind]
            train_set = lgb.Dataset(x_train, y_train, categorical_feature = categoricals)
            val_set = lgb.Dataset(x_val, y_val, categorical_feature = categoricals)

            model = lgb.train(params, train_set, num_boost_round = 100000, early_stopping_rounds = 100, 
                              valid_sets=[train_set, val_set], verbose_eval = 500)
            oof_pred[val_ind] = model.predict(x_val)
        loss_score = cohen_kappa_score(reduce_train[target], np.argmax(oof_pred, axis = 1), weights = 'quadratic')
        print('Our new cohen kappa score is : {}'.format(loss_score))
        if loss_score > score:
            print('Feature {} is usefull, adding feature to usefull_new_features_list'.format(i))
            usefull_new_features.append(i)
            score = loss_score
        else:
            print('Feature {} is useless'.format(i))
        gc.collect()
    print('The best features are: ', usefull_new_features)
    print('Our best cohen kappa score is : ', score)

    return usefull_features + usefull_new_features

In [7]:
def run_lgb(reduce_train, reduce_test, usefull_features):
    kf = StratifiedKFold(n_splits=10, shuffle = True, random_state = 42)
    target = 'accuracy_group'
    oof_pred = np.zeros((len(reduce_train), 4))
    y_pred = np.zeros((len(reduce_test), 4))
    for fold, (tr_ind, val_ind) in enumerate(kf.split(reduce_train, reduce_train[target])):
        print('Fold {}'.format(fold + 1))
        x_train, x_val = reduce_train[usefull_features].iloc[tr_ind], reduce_train[usefull_features].iloc[val_ind]
        y_train, y_val = reduce_train[target][tr_ind], reduce_train[target][val_ind]
        train_set = lgb.Dataset(x_train, y_train, categorical_feature=categoricals)
        val_set = lgb.Dataset(x_val, y_val, categorical_feature=categoricals)

        params = {
            'learning_rate': 0.07,
            'metric': 'multiclass',
            'objective': 'multiclass',
            'num_classes': 4,
            'feature_fraction': 0.7,
            'subsample': 0.7,
            'n_jobs': -1,
            'seed': 50,
            'max_depth': 10
        }

        model = lgb.train(params, train_set, num_boost_round = 1000000, early_stopping_rounds = 100, 
                          valid_sets=[train_set, val_set], verbose_eval = 100)
        oof_pred[val_ind] = model.predict(x_val)
        y_pred += model.predict(reduce_test[usefull_features]) / 10
    loss_score = cohen_kappa_score(reduce_train[target], np.argmax(oof_pred, axis = 1), weights = 'quadratic')
    result = pd.Series(np.argmax(oof_pred, axis = 1))
    print('Our oof cohen kappa score is: ', loss_score)
    print(result.value_counts(normalize = True))
    return y_pred

def predict(reduce_test, sample_submission, y_pred):
    sample_submission['accuracy_group'] = y_pred.argmax(axis = 1)
    sample_submission.to_csv('submission.csv', index = False)
    print(sample_submission['accuracy_group'].value_counts(normalize = True))

In [8]:
# read data
train, test, train_labels, specs, sample_submission = read_data()
# get usefull dict with maping encode
train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code = encode_title(train, test, train_labels)
# tranform function to get the train and test set
reduce_train, reduce_test, categoricals = get_train_and_test(train, test)

Reading train.csv file....
Training.csv file have 11341042 rows and 11 columns
Reading test.csv file....
Test.csv file have 1156414 rows and 11 columns
Reading train_labels.csv file....
Train_labels.csv file have 17690 rows and 7 columns
Reading specs.csv file....
Specs.csv file have 386 rows and 3 columns
Reading sample_submission.csv file....
Sample_submission.csv file have 1000 rows and 2 columns


In [9]:
features = reduce_train.loc[(reduce_train.sum(axis=1) != 0), (reduce_train.sum(axis=0) != 0)].columns # delete useless columns
features = [x for x in features if x not in ['accuracy_group', 'installation_id']]
reduce_train[features]

,Clip,Activity,Assessment,Game,acc_Chest Sorter (Assessment),acc_Cart Balancer (Assessment),acc_Cauldron Filler (Assessment),acc_Bird Measurer (Assessment),acc_Mushroom Sorter (Assessment),2050,4100,4230,5000,4235,2060,4110,5010,2070,2075,2080,2081,2083,3110,4010,3120,3121,4020,4021,4022,4025,4030,4031,3010,4035,4040,3020,3021,4045,2000,4050,2010,2020,4070,2025,2030,4080,2035,2040,4090,4220,4095,5b49460a,bdf49a58,15ba1109,30df3273,14de4c5d,31973d56,d2278a3b,a8efe47b,85de926c,beb0a7b9,2dc29e21,cf7638f3,3ee399c3,37ee8496,ab3136ba,a52b92d5,c6971acf,8af75982,795e4a37,6bf9e3e1,df4940d3,f7e47413,9e4c8c7b,f32856e4,67aa2ada,b2e5b0f1,1cc7cfca,90ea0bac,3a4be871,0d18d96c,a5be6304,2230fab4,c7fe2a55,93edfe2e,d88ca108,e9c52111,26a5a3dd,e4d32835,91561152,9e34ea74,a1bbe385,30614231,f54238ee,c1cac9a2,0ce40006,9e6b7fb5,1bb5fbdb,5f5b2617,7040c096,6f8106d9,7da34a02,84b0e0c8,b120f2ac,a76029ee,86ba578b,9de5e594,28520915,d38c2fd7,df4fe8b6,16667cc5,0db6d71d,a8876db3,8d84fa81,73757a5e,7ad3efc6,87d743c1,99ea62f3,3d8c61b0,93b353f2,7d093bf9,6cf7d25c,1cf54632,d3640339,45d01abe,cfbd47c8,160654fd,7ab78247,b80e5e84,9ee1c98c,51102b85,0d1da71f,47026d5f,a6d66e51,56817e2b,46b50ba8,155f62a4,3edf6747,26fd2d99,d45ed6a1,3bf1cf26,c0415e5c,f5b8c21a,6f4adc4b,38074c54,77c76bc5,d2e9262e,48349b14,ea296733,ac92046e,6043a2b4,77ead60d,56cd3b43,1996c610,1575e76c,1375ccb7,ad2fc29c,0a08139c,d06f75b5,c277e121,2a512369,c54cf6c5,3afde5dd,cf82af56,b7530680,0330ab6a,15a43e5b,262136f4,9ed8f6da,46cd75b4,a592d54e,a2df0760,2fb91ec1,5de79a6a,ad148f58,f71c4741,16dffff1,7423acbc,3dfd4aa4,f56e0afc,bbfe0445,2dcad279,e04fb33d,2ec694de,56bcd38d,74e5f8a7,b74258a0,e5734469,06372577,3bb91dda,99abe2bb,44cb4907,01ca3a3c,15eb4a7d,89aace00,cb6010f8,65a38bf7,3323d7e9,3babcb9b,6c517a88,4b5efe37,ecc6157f,a16a373e,d185d3ea,ab4ec3a4,565a3990,c7f7f0e1,bb3e370b,28a4eb9a,6d90d394,19967db1,90efca10,1340b8d7,8ac7cce4,d3268efa,37c53127,d122731b,a44b10dc,49ed92e9,eb2c19cd,5be391b5,47f43a44,e3ff61fb,e64e2cfd,ca11f653,83c6c409,8f094001,832735e1,c2baf0bd,b012cd7f,736f9581,3ccd3f02,6aeafed4,792530f8,3d63345e,499edb7c,b5053438,250513af,3b2048ee,c51d8688,709b1251,2b058fe3,5e812b27,c7128948,7ec0c298,9c5ef70c,b7dc8128,4c2ec19f,25fa8af4,d51b1749,e57dd7af,15f99afc,bfc77bd6,7fd1ac25,d9c005dd,a8a78786,0413e89d,02a42007,5d042115,29bdd9ba,1beb320a,c189aaf2,e080a381,c74f40cd,5290eab1,9b4001e4,363d3849,1c178d24,7f0836bf,51311d7a,1f19558b,37db1c2f,f806dc10,dcb55a27,47efca07,e694a35b,2c4e6db0,5154fc30,4a4c3d21,0086365d,77261ab5,5c3d2b2f,884228c8,c952eb01,4bb2f698,e5c9df6f,907a054b,8b757ab8,b1d5101d,4ef8cdd3,33505eae,119b5b02,37937459,3dcdda7f,fcfdffb6,67439901,daac11b0,58a0de5c,b88f38da,ea321fb1,461eace6,222660ff,05ad839b,a0faea5d,84538528,cdd22e43,27253bdc,7372e1a5,9b01374f,e4f1efe6,392e14df,5c2f29ca,a7640a16,fbaf3456,d2659ab4,3393b68b,1af8be29,562cec5f,5e109ec3,92687c59,9d4e7b25,90d848e0,6077cc36,17113b36,587b5989,f93fc684,c58186bf,e7e44842,e720d930,65abac75,bc8f2793,4e5fc6f5,1b54d27f,857f21c0,e7561dd2,5f0eb72c,8fee50e2,4901243f,f3cd5473,022b4259,d3f1e122,923afab1,e37a2b78,55115cbd,6c930e6e,69fdac0a,abc5811c,29a42aea,5a848010,fd20ea40,76babcde,de26c3a6,d88e8f25,828e68f9,71fe8f75,00c73085,04df9b66,9554a50b,070a5291,4d911100,71e712d8,db02c830,d02b7a8e,29f54413,bd701df8,731c0cbe,363c86c9,acf5c23f,763fc34e,611485c5,f50fc6c1,bd612267,bcceccc6,3bfd1a65,2b9272f4,8d7e386c,63f13dd7,a1192f43,7dfe6d8a,28ed704e,532a2afb,5e3ea25a,88d4a5be,7d5c30a2,cc5087a3,13f56524,a1e4395d,9ce586dd,85d1b0de,598f4598,cb1178ad,ecaab346,a29c5338,28f975ea,08fd73f3,e79f3763,7961e599,4a09ace1,b2dba42b,7cf1bc53,ec138c1c,3bb91ced,3afb49e6,3d0b9317,a5e9da97,6f4bd64e,53c6e11a,9d29771f,895865f3,7525289a,3ddc79c3,6f445b57,ecc36b7f,36fa3ebe,86c924c4,9b23e8ee,f28c589a,dcaede90,2a444e03,4d6737eb,804ee27f,5859dfb6,f6947f54,5348fd84,08ff79ad,6088b756,756e5507,8d748b58,1325467d,Chest Sorter (Assessment),Balancing Act,Bottle Filler (Activity),Chicken Balancer (Activity),Mushroom Sorter (Assessment),Crystal Caves - Level 2,Crystal Caves - Level 1,Cart Balancer (Assessment),Bu

In [10]:
# # next codes are to select only the feature that increase our cohen kappe score
# new_features = ['installation_session_count', 'installation_duration_mean', 'installation_duration_std', 'installation_title_nunique', 'sum_event_code_count', 'installation_event_code_count_mean', 
#                 'installation_event_code_count_std']
# usefull_features = [col for col in features if col not in new_features]
# best_features = run_feature_selection(reduce_train, reduce_test, usefull_features, new_features)

In [11]:
y_pred = run_lgb(reduce_train, reduce_test, features)
predict(reduce_test, sample_submission, y_pred)

Fold 1


/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.780418	valid_1's multi_logloss: 0.95468
[200]	training's multi_logloss: 0.645615	valid_1's multi_logloss: 0.951654
Early stopping, best iteration is:
[140]	training's multi_logloss: 0.720234	valid_1's multi_logloss: 0.950142
Fold 2
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.775382	valid_1's multi_logloss: 0.993893
[200]	training's multi_logloss: 0.643355	valid_1's multi_logloss: 0.993477
Early stopping, best iteration is:
[165]	training's multi_logloss: 0.684974	valid_1's multi_logloss: 0.990497
Fold 3
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.775309	valid_1's multi_logloss: 0.969762
[200]	training's multi_logloss: 0.641566	valid_1's multi_logloss: 0.975552
Early stopping, best iteration is:
[118]	training's multi_logloss: 0.74668	valid_1's multi_logloss: 0.968858
Fold 4
Training until 